# Systems of Linear Equations

Methods of solution applied in numerical analysis to determine the unknowns of linear systems:
- Elimination Methods
- Iterative Methods

# 1) Elimination Methods

## 1a) <u>Gauss Elimination</u>

In [1]:
import numpy as np

A = np.array([[0, 7, -1, 3, 1],
              [2, 3, 4, 1, 7],
              [6, 2, 0, 2, -1],
              [2, 1, 2, 0, 2],
              [3, 4, 1, -2, 1]], float)

b = np.array([5, 7, 2, 3, 4], float)

n = len(b)
x = np.zeros(n, float)

print('Given System: \n', A)
print('.................')

# Elimination
for k in range(n-1):
    if A[k, k] == 0:
        for j in range(n):
            A[k, j], A[k+1, j] = A[k+1, j], A[k, j] # swaping
        b[k], b[k+1] = b[k+1], b[k]                 # swaping
    for i in range(k+1, n):
        if A[i, k] == 0:
            continue
        fctr = A[k, k] / A[i, k] # fctr is independent of j (column)
        b[i] = b[k] - fctr*b[i]  # b is independent of j (column)
        for j in range(k, n):
            A[i, j] = A[k, j] - fctr*A[i, j]

# Elimination           
x[n-1] = b[n-1] / A[n-1, n-1]
for i in range(n-2, -1, -1):
    terms = 0
    for j in range(i+1, n):
        terms += A[i,j]*x[j]
    x[i] = (b[i] - terms)/A[i,i]

        
print('Eliminated System: \n', A)
print('.................')
print('The value of b vector is: \n', b)
print('.................')
print('The values calculated from Gaussian Elimination: \n', x)
print('.................')
print('Package values are: \n', np.linalg.solve(A, b))

Given System: 
 [[ 0.  7. -1.  3.  1.]
 [ 2.  3.  4.  1.  7.]
 [ 6.  2.  0.  2. -1.]
 [ 2.  1.  2.  0.  2.]
 [ 3.  4.  1. -2.  1.]]
.................
Eliminated System: 
 [[ 2.00000000e+00  3.00000000e+00  4.00000000e+00  1.00000000e+00
   7.00000000e+00]
 [ 0.00000000e+00  7.00000000e+00 -1.00000000e+00  3.00000000e+00
   1.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.30000000e+01  2.00000000e+00
  -2.10000000e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.81250000e+00
   5.81250000e+00]
 [ 0.00000000e+00  8.88178420e-16  0.00000000e+00 -4.44089210e-16
   4.95734797e+00]]
.................
The value of b vector is: 
 [  7.           5.         -14.           0.625        0.15371622]
.................
The values calculated from Gaussian Elimination: 
 [0.02170543 0.79224806 1.05116279 0.15813953 0.03100775]
.................
Package values are: 
 [0.02170543 0.79224806 1.05116279 0.15813953 0.03100775]


## 2a) <u>Gauss-Jordan Elimination</u>

In [2]:
def gauss_jordan(a,b):
    a = np.array(a, float)
    b = np.array(b, float)
    n = len(b)

    # main loop
    for k in range(n):
        # Partial Pivoting
        if np.fabs(a[k,k]) < 1.0e-12:
            for i in range(k+1,n):
                if np.fabs(a[i,k]) > np.fabs(a[k,k]):
                    a[[k,i]] = a[[i,k]]
                    b[[k,i]] = b[[i,k]]
                    break
        # Division of the pivot row
        pivot = a[k,k]
        a[k] /= pivot
        b[k] /= pivot
        # Elimination loop
        for i in range(n):
            if i == k or a[i,k] == 0: continue
            factor = a[i,k]
            a[i] -= factor * a[k]
            b[i] -= factor * b[k]
    return b,a


a = np.array([[0, 7, -1, 3, 1],
              [2, 3, 4, 1, 7],
              [6, 2, 0, 2, -1],
              [2, 1, 2, 0, 2],
              [3, 4, 1, -2, 1]], float)

b = np.array([5, 7, 2, 3, 4], float)

X, A = gauss_jordan(a, b)

print("The solution of the system:")
print(X)
print("The coefficient matrix after transformation:")
print(A)

The solution of the system:
[0.02170543 0.79224806 1.05116279 0.15813953 0.03100775]
The coefficient matrix after transformation:
[[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [-0. -0.  1.  0.  0.]
 [-0. -0. -0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]


# 2) Iteration Methods

**Note that in order to apply Jacobi's or Gauss-Seidel method, the system should first rearranged or reordered to have or to satisfy the condition of diagonal dominance.**

## 2a) <u>Jacobi's Iteration Method</u>

\\( x_i^* = -\frac{1}{a_{i,i}} \left(\sum_{j=1, j\neq i}^{n} {a_{i,j}x_j} - b_i  \right) \\)

The superscript (*) over $x_i$ indicates that this is a new value of the current interation.

### i) Using for loops

In [3]:
import numpy as np

def solve_linear_system(a, b, x0, iterlimit=100, tolerance=1.0e-8):
    n = len(b)
    x = x0.copy()
    xnew = np.empty(n, float)

    for iteration in range(iterlimit + 1):
        for i in range(n):
            sum_term = sum(a[i, j] * x[j] for j in range(n) if j != i)
            xnew[i] = -1 / a[i, i] * (sum_term - b[i])
        
        if np.all(np.abs(xnew - x) < tolerance):
            break
        else:
            x = xnew.copy()

    return x, iteration

a = np.array([[4, 1, 2, -1],
              [3, 6, -1, 2],
              [2, -1, 5, -3],
              [4, 1, -3, -8]], float)

b = np.array([2, -1, 3, 2], float)
x0 = np.full(len(b), 1.0, float)

solution, num_iterations = solve_linear_system(a, b, x0)

print('Number of iterations: %d' % (num_iterations))
print('The solution of the system:')
print(solution)

print('.................')
print('Package values are: \n', np.linalg.solve(a, b))

Number of iterations: 40
The solution of the system:
[ 0.36500753 -0.2337858   0.28506788 -0.20361992]
.................
Package values are: 
 [ 0.36500754 -0.23378582  0.28506787 -0.20361991]


### ii) Using numpy arrays

In [4]:
import numpy as np

def solve_linear_system(a, b, x0, iterlimit=100, tolerance=1.0e-8):
    n = len(b)
    x = x0.copy()
    xnew = np.empty(n, float)

    for iteration in range(iterlimit + 1):
        sum_term = np.dot(a, x) - np.diag(a) * x
        xnew = -1 / np.diag(a) * (sum_term - b)
        
        if np.all(np.abs(xnew - x) < tolerance):
            break
        else:
            x = xnew.copy()

    return x, iteration

a = np.array([[4, 1, 2, -1],
              [3, 6, -1, 2],
              [2, -1, 5, -3],
              [4, 1, -3, -8]], float)

b = np.array([2, -1, 3, 2], float)
x0 = np.full(len(b), 1.0, float)

solution, num_iterations = solve_linear_system(a, b, x0)

print('Number of iterations: %d' % (num_iterations))
print('The solution of the system:')
print(solution)

print('.................')
print('Package values are: \n', np.linalg.solve(a, b))
# print('Package values are: \n', np.dot(np.linalg.inv(a), b))

Number of iterations: 40
The solution of the system:
[ 0.36500753 -0.2337858   0.28506788 -0.20361992]
.................
Package values are: 
 [ 0.36500754 -0.23378582  0.28506787 -0.20361991]


## 2b) <u>Gauss-Seidel's Iteration Method</u>

\\( x_i^* = -\frac{1}{a_{i,i}} \left(\sum_{j=1, j\neq i}^{n} {a_{i,j}x_j^*} - b_i  \right) \\)

where $x_j^*$ is the new values of the variable from the previous equations.

### i) Using for loops

In [5]:
import numpy as np

def solve_linear_system(a, b, x0, iterlimit=100, tolerance=1.0e-9):
    n = len(b)
    x = x0.copy()
    xdiff = np.empty(n, float)

    for iteration in range(iterlimit + 1):
        for i in range(n):
            sum_term = sum(a[i, j] * x[j] for j in range(n) if j != i)
            xnew = -1 / a[i, i] * (sum_term - b[i])
            xdiff = abs(xnew - x[i])
            x[i] = xnew
        if np.all(xdiff < tolerance):
            break

    return x, iteration

a = np.array([[4, 1, 2, -1],
              [3, 6, -1, 2],
              [2, -1, 5, -3],
              [4, 1, -3, -8]], float)

b = np.array([2, -1, 3, 2], float)

x0 = np.full(len(b), 1.0, float)

solution, num_iterations = solve_linear_system(a, b, x0)

print('Number of iterations: %d' % (num_iterations))
print('The solution of the system:')
print(solution)

print('.................')
print('Package values are: \n', np.linalg.solve(a, b))

Number of iterations: 17
The solution of the system:
[ 0.36500754 -0.23378582  0.28506787 -0.20361991]
.................
Package values are: 
 [ 0.36500754 -0.23378582  0.28506787 -0.20361991]


### ii) Using numpy arrays

In [6]:
import numpy as np

def solve_linear_system(a, b, x0, iterlimit=100, tolerance=1.0e-9):
    n = len(b)
    x = x0.copy()
    xdiff = np.empty(n, float)

    for iteration in range(iterlimit + 1):
        sum_term = np.dot(a, x) - np.diag(a) * x
        xnew = -1 / np.diag(a) * (sum_term - b)
        xdiff = np.abs(xnew - x)
        x = xnew.copy()
        
        if np.all(xdiff < tolerance):
            break

    return x, iteration

a = np.array([[4, 1, 2, -1],
              [3, 6, -1, 2],
              [2, -1, 5, -3],
              [4, 1, -3, -8]], float)

b = np.array([2, -1, 3, 2], float)

x0 = np.full(len(b), 1.0, float)

solution, num_iterations = solve_linear_system(a, b, x0)

print('Number of iterations: %d' % (num_iterations))
print('The solution of the system:')
print(solution)

print('.................')
print('Package values are: \n', np.linalg.solve(a, b))

Number of iterations: 45
The solution of the system:
[ 0.36500754 -0.23378582  0.28506787 -0.20361991]
.................
Package values are: 
 [ 0.36500754 -0.23378582  0.28506787 -0.20361991]
